In [167]:
import math

import pandas as pd
from neupy import algorithms
import numpy as np
import json
from tqdm import tqdm
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from numpy import sqrt
import math

In [168]:
def ret(n, n_1):
    return (n_1 - n) / n


NDTR = 2
std = 0.0003

# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-1m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
# df.columns = ['price']
df['ret_c'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
df['ret_o'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)
df['ret'] = (df['close'].shift(-1) - df['close']) / df['close']
df = df.iloc[1:NDTR + 1001]

In [169]:
df

,timestamp,open,high,low,close,vol,ret_c,ret_o,ret
1,1643760060000,38724.92,38786.36,38714.47,38763.09,14.60847,0.001051,0.000784,-0.000417
2,1643760120000,38763.08,38768.18,38715.64,38746.91,28.64245,-0.000417,0.000985,-0.000487
3,1643760180000,38746.90,38754.50,38725.00,38728.05,15.48239,-0.000487,-0.000417,-0.000183
4,1643760240000,38728.05,38728.05,38714.00,38720.95,11.37676,-0.000183,-0.000486,-0.001162
5,1643760300000,38720.95,38730.46,38675.93,38675.95,15.71739,-0.001162,-0.000183,-0.001328
...,...,...,...,...,...,...,...,...,...
998,1643819880000,37196.08,37196.09,37121.33,37122.28,28.10698,-0.001984,0.000513,0.000344
999,1643819940000,37122.28,37140.49,37100.00,37135.05,42.44593,0.000344,-0.001984,0.001048
1000,1643820000000,37135.05,37173.99,37126.04,37173.98,16.25093,0.001048,0.000344,0.000188
1001,1643820060000,37173.98,37199.99,37171.27,37180.97,19.89329,0.000188,0.001048,0.000045


In [170]:
predictions = [0] * NDTR
for i in np.arange(NDTR, NDTR + 1000):
    small_df = df.iloc[i: i + NDTR + 1].copy(deep=True)[['ret_c', 'ret_o', 'ret']].reset_index(drop=True)
    x_train = small_df.iloc[0:NDTR][['ret_c', 'ret_o']].values
    y_train = small_df['ret'].iloc[0:NDTR].values
    grnn = algorithms.GRNN(std=std, verbose=False)
    grnn.train(x_train, y_train)
    if len(small_df) > NDTR:
        x_predict = small_df.iloc[NDTR][['ret_c', 'ret_o']].values
        prediction = grnn.predict([x_predict])
        if math.isnan(prediction):
            prediction = 0
        else:
            prediction = prediction[0][0]
        predictions.append(prediction)
    else:
        predictions.append(0)

df['ret_p'] = predictions

In [171]:
df

,timestamp,open,high,low,close,vol,ret_c,ret_o,ret,ret_p
1,1643760060000,38724.92,38786.36,38714.47,38763.09,14.60847,0.001051,0.000784,-0.000417,0.000000
2,1643760120000,38763.08,38768.18,38715.64,38746.91,28.64245,-0.000417,0.000985,-0.000487,0.000000
3,1643760180000,38746.90,38754.50,38725.00,38728.05,15.48239,-0.000487,-0.000417,-0.000183,-0.000186
4,1643760240000,38728.05,38728.05,38714.00,38720.95,11.37676,-0.000183,-0.000486,-0.001162,-0.001328
5,1643760300000,38720.95,38730.46,38675.93,38675.95,15.71739,-0.001162,-0.000183,-0.001328,0.000776
...,...,...,...,...,...,...,...,...,...,...
998,1643819880000,37196.08,37196.09,37121.33,37122.28,28.10698,-0.001984,0.000513,0.000344,0.001048
999,1643819940000,37122.28,37140.49,37100.00,37135.05,42.44593,0.000344,-0.001984,0.001048,0.000188
1000,1643820000000,37135.05,37173.99,37126.04,37173.98,16.25093,0.001048,0.000344,0.000188,0.000052
1001,1643820060000,37173.98,37199.99,37171.27,37180.97,19.89329,0.000188,0.001048,0.000045,0.000000


In [172]:
# copy_df = df.groupby(df.index // 6)['ret'].sum()
# negatives = list(filter(lambda x: x < 0, copy_df.values))
# positives = list(filter(lambda x: x > 0, copy_df.values))
#
# fig = make_subplots()
# fig.add_trace(go.Scatter(x=[*range(len(copy_df))], y=copy_df.values, mode='lines+markers'))
# fig.add_trace(go.Scatter(x=[*range(len(copy_df))], y=[np.array(negatives).mean()] * len(copy_df), name="Average of negatives"))
# fig.add_trace(go.Scatter(x=[*range(len(copy_df))], y=[np.array(positives).mean()] * len(copy_df), name="Average of positives"))
#
# fig.show()

In [173]:
buy_top_thr = 0
buy_bot_thr = -0.001
sell_top_thr = 0.001
sell_bot_thr = 0

In [174]:
# # buys
CANDLES_TO_LOOK = 6

buys = [0] * (NDTR + CANDLES_TO_LOOK)
sells = [0] * (NDTR + CANDLES_TO_LOOK)
timeout = 0
for i in np.arange(NDTR + CANDLES_TO_LOOK, NDTR + 1000):
    small_df = df.iloc[i + (NDTR - CANDLES_TO_LOOK): i + (NDTR + 1)].copy(deep=True).reset_index(
        drop=True)  # me traigo ultimas CANDLES_TO_LOOK velas + actual
    ret_sum = small_df.iloc[:CANDLES_TO_LOOK]['ret'].sum()  # ultimas CANDLES_TO_LOOK velas
    if len(small_df) > CANDLES_TO_LOOK:
        actual = small_df['ret_p'].iloc[3]
    else:
        actual = 0

    if timeout == 10:
        timeout = 0
    elif timeout != 0:
        timeout += 1

    if ret_sum < buy_bot_thr and actual > buy_top_thr and timeout == 0:
        buys.append(1)
        timeout = 1
    else:
        buys.append(0)

    if ret_sum > sell_top_thr and actual < sell_bot_thr:
        sells.append(1)
        timeout=1
    else:
        sells.append(0)

df['buy'] = buys
df['sell'] = sells
#
# print('buys=', len(df[df['buy'] == 1]))
# print('sells=', len(df[df['sell'] == 1]))
#
# fig = make_subplots()
# fig.add_trace(go.Scatter(x=df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)), y=df['close'],
#                          name="Close price in time"))
# fig.add_trace(go.Scatter(x=df[df['buy'] == 1]['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)),
#                          y=df[df['buy'] == 1]['close'], name="Buys", mode="markers"))
# fig.add_trace(go.Scatter(x=df[df['sell'] == 1]['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)),
#                          y=df[df['sell'] == 1]['close'], name="Sells", mode="markers"))
# fig.update_layout(title="Probable buys and sells for NDTR=" + str(NDTR) + " and std=" + str(std))
# fig.show()

In [175]:
# only one trade at a time
copy_df = df.copy(deep=True)
rets = []
dates = []
index_of_buy = []
index_of_sell = []
for i in range(len(copy_df)):
    row = copy_df.iloc[i]
    buy_price = row['close']
    if row['buy'] == 1:
        already_selled = False
        for j in np.arange(i + 1, len(copy_df)):
            if not already_selled:
                possible_sell = copy_df.iloc[j]
                sell_price = possible_sell['close']
                if possible_sell['sell'] == 1 and j not in index_of_sell:
                    index_of_sell.append(j)
                    index_of_buy.append(i)
                    i = j + 1

                    rets.append(ret(buy_price, sell_price))
                    dates.append(datetime.fromtimestamp(possible_sell['timestamp'] / 1000))
                    already_selled = True
                    break

rets_cumsum = np.array(rets).cumsum()
fig = make_subplots()
fig.add_trace(go.Scatter(x=dates, y=rets_cumsum, mode='lines+markers'))
fig.update_layout(title="Cumsum rets in time. Amount of trades=" + str(len(index_of_sell)), yaxis_tickformat=".00%")
fig.show()

fig = make_subplots()
fig.add_trace(go.Scatter(x=df['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)), y=df['close'],
                         name="Close price in time"))
fig.add_trace(go.Scatter(x=copy_df[copy_df['buy'] == 1]['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)),
                         y=copy_df[copy_df['buy'] == 1]['close'], name="Buys", mode="markers"))
fig.add_trace(go.Scatter(x=copy_df[copy_df['sell'] == 1]['timestamp'].apply(lambda x: datetime.fromtimestamp(x / 1000)),
                         y=copy_df[copy_df['sell'] == 1]['close'], name="Sells", mode="markers"))
fig.update_layout(
    title="Probable buys and sells for NDTR=" + str(NDTR) + " and std=" + str(std) + " | Amount of trades=" + str(
        len(index_of_sell)))
fig.show()